In [1]:
import sys

#Due to leftoevererrors in Nathan's python installation, some cleaning up occurs here
#sys.path.append("./code/")
#sys.path.remove('/usr/local/lib/python2.7/site-packages') 


import matplotlib.pyplot as plt
%matplotlib inline
import copy
import datetime
import pickle
import gzip
import os,glob
import time
import numpy as np
import pandas as pd
import operator
import random
from  scipy import stats



from scipy.stats import norm
from sklearn.neighbors import KernelDensity


import itertools
#sys.path


import regex as re
import datetime
import math
import time
import itertools

import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
import plotly.figure_factory as ff

import plotly
plotly.tools.set_credentials_file(username='juliettapc', api_key='nM6iUdx6dGaOiPXQTwpP')   # go to: https://plot.ly/settings/api#/   for a new key if needed

########## to be able to plot offline (without sending the plots to the plotly server every time)
import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
################



from IPython.core.display import display,HTML
display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser


In [2]:
                                  

####  NEW FILE INCLUDING EARLY CITATIONS OF YOUNG REFERENCES:   ../data/df_reference_cite_plos_no_self-cit_one_ref_per_sect_ONLY_ARTICLES_early_cit.pkl
                                            
#%time df_merged = pickle.load(open('../data/df_reference_cite_plos_merged_simplified_added_more_columns_no_self-cit_one_ref_per_sect_ONLY_ARTICLES.pkl', 'rb'))



%time df_merged = pickle.load(open('../data/df_reference_cite_plos_no_self-cit_one_ref_per_sect_ONLY_ARTICLES_early_cit_and_after_accretion_time.pkl', 'rb'))
print ("done loading pickles", df_merged.shape)




df_merged = df_merged[df_merged['cite_count'] != -1]   # i dont know why, but there are 7 occurrences with value -1



plos_df = df_merged.drop_duplicates(subset=['paper_UT'])
print (plos_df.shape)



df_merged.head()

CPU times: user 4.3 s, sys: 790 ms, total: 5.09 s
Wall time: 5.09 s
done loading pickles (5787634, 34)
(156558, 34)


,occurence,reference_UT,reference_rank,regex_sect_index,cite_count,ref_pub_year,paper_cite_count,plos_pub_year,sect_char_pos,sect_char_total,...,plos_article_type,num_cit_young_ref_by2009,num_cit_young_ref_by2010,num_cit_young_ref_by2011,num_cit_young_ref_by2012,num_cit_young_ref_by2013,num_cit_young_ref_by2009after8,num_cit_young_ref_by2008after8,num_cit_young_ref_by2010after8,num_cit_young_ref_by2007after8
0,1,A1995QY75100004,1,0,60.0,1995.0,2,2013.0,139,4029,...,@ Article,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,A1995QY75100004,1,3,60.0,1995.0,2,2013.0,494,5398,...,@ Article,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,000263911400006,3,0,5.0,2009.0,2,2013.0,142,4029,...,@ Article,0.0,0.0,NaN,NaN,NaN,5.0,5.0,5.0,5.0
5,1,000289279600018,4,0,29.0,2011.0,2,2013.0,269,4029,...,@ Article,0.0,0.0,0.0,3.0,NaN,39.0,39.0,39.0,39.0
6,3,000289279600018,4,3,29.0,2011.0,2,2013.0,3844,5398,...,@ Article,0.0,0.0,0.0,3.0,NaN,39.0,39.0,39.0,39.0


In [3]:

#  annotated heatmap plot for median publication year (OR CITATIONS) of the references used in the different sections, and separating by citation category of the plos


##### FIGURE 2B  AND FIGURE 3A AND FIGURE 3B and the corresponding pairwise comparisons


dict_group_subset_data={}
dict_group_quantiles_size={}

######### in this cell I SELECT the data i want to plot (by multiple criteria), as well as the variable that will encode with color:
######### ######### ######### ######### ######### ######### ######### 


v1_string = 'cite_count'   #      cite_count    diff_year_plos_ref 
       
  
string_references_age = "old"#old"   #"#old"  # young # all   for the selection of what references i include
  
  
  
top_space = 150
if v1_string ==  'cite_count'  :
    colorbar_string = 'Citations'
    if string_references_age == "old" :
        colorbar_string = ''
else:
    colorbar_string = 'Age [yr]'
    top_space = 100
    text_abc = '(b)'

years=[2011] 



#years=[2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017] 



list_q=[0.3,0.6,.9,.99,1]    # for the percentile sections for number of citations of the PLOS papers
 
    
string_filtering_x = 'paper_cite_count'   # bins by plos' citations on the x-axis ###      
  
    
      
      
    
    
    
    

string_isolated_ref = ""   #""   #"  #""   # 0  or 1 (or empty string, to include all ref)
#list_strings = [1,0]
#for  string_isolated_ref  in list_strings:
    
    


string_self_ref =0         #""      # ""   #1   # 0  or 1 (or empty string, to include all ref)   OJO!!! THIS NEW FILE DOES NOT INCLUDE SELF-CITATIONS TO BEGING WITH




######### plos ONE categories. 
string_code_categ="" #  ojo!!! the codes are strings, not integers. if i want to include multiple subjects:  "1 2 8"


#list_strings=['0', '1', '4', '5', '7', '8', '2 3','10 6 9']
#for string_code_categ in list_strings:


#   Biology and Live Sciences;   Computational Sciences;   Engineering;   Medicine;   Physical Sciences;   Research and Analysis; \
#  Earth Sciences and Ecology;   Social Sciences, Political Sciences and People & Places



#  '0': 'Biology and life sciences'             6,032,537 --
#  '1': 'Computer and information sciences'     1,207,799 --
#  '10': 'Social sciences'                      755,899 --
#  '2': 'Earth sciences'                        533,155 --
#  '3': 'Ecology and environmental sciences'    624,142 --
#  '4': 'Engineering and technology'            382,247 --
#  '5': 'Medicine and health sciences'          4,535,926  -- 
#  '6': 'People and places'                     691,523 --
#  '7': 'Physical sciences'                     2,100,827 --
#  '8': 'Research and analysis methods'         3,871,470 --
#  '9': 'Science policy'                        43,360 --


######### plos journals 
string_journal=""#   PLOS ONE"

#list_strings=['PLOS MED', 'PLOS BIOL', 'PLOS COMPUT', 'PLOS PATHOG', 'PLO NE TR D', 'PLOS GENET', 'PLOS ONE']


#for string_journal in list_strings:

    # PLOS ONE       6,367,070
    # PLOS GENET      149,923
    # PLO NE TR D     138,289   # (neglected tropical diseases)
    # PLOS PATHOG     109,803
    # PLOS COMPUT      77,924
    # PLOS BIOL        56,754
    # PLOS MED         24,506



#PLOS Medicine, PLOS Biol-ogy, PLOS Computational Biology, PLOS Pathology, PLOS Neglected Tropical Diseases, PLOSGenetics, and PLOS ONE


######### WoS subject categories. 
string_plos_field=""#['D CU BIOLOGY']"

# ['D RO MULTIDISCIPLINARY SCIENCES']                                                                                                       4464540
# ['D CU BIOLOGY']                                                                                                                          1055045
# ['D RO MULTIDISCIPLINARY SCIENCES', 'D CU BIOLOGY']                                                                                        847485
# ['D KM GENETICS & HEREDITY']                                                                                                               149923
# ['D YU TROPICAL MEDICINE', 'D TI PARASITOLOGY']                                                                                            138289
# ['D ZE VIROLOGY', 'D QU MICROBIOLOGY', 'D TI PARASITOLOGY']                                                                                109803
# ['D CO BIOCHEMICAL RESEARCH METHODS', 'D MC MATHEMATICAL & COMPUTATIONAL BIOLOGY']                                                          77687
# ['D CQ BIOCHEMISTRY & MOLECULAR BIOLOGY', 'D CU BIOLOGY']                                                                                   56754
# ['D PY MEDICINE, GENERAL & INTERNAL']                                                                                                       24506
# ['D CO BIOCHEMICAL RESEARCH METHODS', 'D MC MATHEMATICAL & COMPUTATIONAL BIOLOGY', 'D PO MATHEMATICS, INTERDISCIPLINARY APPLICATIONS']        237











print ("original size:",df_merged.shape)








##### preselection by plos year
print (years)
preselection_df = df_merged[df_merged['plos_pub_year'].isin(years)]  
print ("size of preselection1 (by plos years):",preselection_df.shape)





#### i remove self-citations
if (string_self_ref==0) or  ( string_self_ref == 1 ): 
    preselection_df = preselection_df[preselection_df['self_citation']== string_self_ref ]  
    if string_self_ref ==0:
        string_self_ref = ", no self-cit"
    elif string_self_ref ==1:
        string_self_ref = ", only self-cit"





######### preselection by isolated or group references:
if (string_isolated_ref==0) or  ( string_isolated_ref == 1 ): 
    preselection_df0 = preselection_df[preselection_df['isolated_citation']== string_isolated_ref ]  

    if string_isolated_ref ==0:
        string_isolated_ref = ", group ref"
    elif string_isolated_ref ==1:
        string_isolated_ref = ", isolated ref"
else:    
    preselection_df0 = preselection_df   
    print ("size of preselection1 (by isolated/group ref):",preselection_df0.shape, string_isolated_ref)








######### preselection by plos ONE subject category:
if string_code_categ=="": 
    preselection_df111 = preselection_df0
else:    
    if " " not in string_code_categ:  # to include one single category
        preselection_df111 = preselection_df0[preselection_df0['categ_codes'].str.contains(string_code_categ)]        
        string_code_categ = " "+dict_code_categ[string_code_categ]  

    else:  # if multiple codes-categories
        list_codes = string_code_categ.split(" ")
        print (list_codes)

        if len(list_codes) >= 2:              
            preselection_df111 = preselection_df0[ preselection_df0['categ_codes'].str.contains('|'.join(list_codes)) ]  # to look for partial matches from a list of strings!!!!!


        string_code_categ = "" 
        for code in list_codes:
            string_code_categ += "-"+dict_code_categ[code] 


    print (" size of preselection (by plos ONE subject category):",preselection_df111.shape, string_code_categ)








######### preselection by plos journal:
if string_journal=="": 
    preselection_df1 = preselection_df111
else:    
    preselection_df1 = preselection_df111[preselection_df111['plos_j1']== string_journal ]  
print (" size of preselection2 (by plos journal):",preselection_df1.shape, string_journal)







######### preselection by plos field:
if string_plos_field=="": 
    preselection_df2 = preselection_df1
else:    
    preselection_df2 = preselection_df1[preselection_df1['plos_field']== string_plos_field ]  
print (" size of preselection2 (by plos field):",preselection_df2.shape, string_plos_field)




preselection_df3 = preselection_df2


N_plos=len(preselection_df3.paper_UT.unique())        ## this values are overall, for the title   
N_all=len(preselection_df3)



fig_font_colors=''



if v1_string ==  'cite_count'  or       v1_string ==  'log_num_cit_ref'   or v1_string == 'log2_num_cit_ref':



    string_age_selection=''

    ##### preselection only young/old references:        
    if string_references_age == "young":
        time_window = 1
        string_age_selection="only young references from >="+ str((min(years)-time_window))
        preselection_df3 = preselection_df2[preselection_df2['ref_pub_year'] >= (min(years)-time_window) ]   
        print ("  size of preselection3 (only young references):",preselection_df3.shape, string_age_selection)

    elif string_references_age == "old":
        time_window = 10
        string_age_selection="only old references from <="+str((min(years)-time_window))
        preselection_df3 = preselection_df2[preselection_df2['ref_pub_year'] <= (min(years)-time_window) ]   
        print ("  size of preselection3 (only young references):",preselection_df3.shape,string_age_selection )

    else:
        string_age_selection="young&old"       
        print ("  No preselection by age of references:",preselection_df3.shape )




    N_plos=len(preselection_df3.paper_UT.unique())        ## this values are overall, for the title   
    N_all=len(preselection_df3)




   # fig_colorscale = [[0, '#ffece6'], [1, '#ff0000']]   # if i give it a min and a max colors in HEX, it creates a gradient from one to another
    fig_font_colors = ['#ff0000', '#ffece6']      # same for the annotation of the boxes (to make sure they are readable)

    
#     factor_color_rescale =.3   

#     fig_colorscale=[[0.0*factor_color_rescale, '#ffffff'],\
#                            [0.1*factor_color_rescale, '#d9f2d9'],\
#                            [0.2*factor_color_rescale, '#c6ecc6'],\
#                            [0.3*factor_color_rescale, '#b3e6b3'],\
#                            [0.4*factor_color_rescale, '#8cd98c'], \
#                            [0.5*factor_color_rescale, '#66cc66'], \
#                            [0.6*factor_color_rescale, '#53c653'], \
#                            [0.7*factor_color_rescale, '#40bf40'],\
#                            [0.75*factor_color_rescale, '#39ac39'],\
#                            [0.8*factor_color_rescale, '#339933'],\
#                            [0.85*factor_color_rescale, '#2d862d'],\
#                            [0.9*factor_color_rescale, '#267326'],\
#                            [1.0, '#000000']]
   
    
    
    


    fig_colorscale = "Reds"
    fig_title_plot = "Median (final) number of citations of references in "+string_journal+string_plos_field+"<br> papers from "+str(years)+", "+string_age_selection+string_isolated_ref+string_self_ref+string_code_categ+"<br>Number of occurrences: "+str(N_all)#+",   Number plos: "+str(N_plos)
    fig_filename =   '../plots/annotated-heatmap_median_citations_of_references_for_sections_and_fract_subsection_by_citations_of_plos'

    if  v1_string ==  'log_num_cit_ref' :
        fig_title_plot = "Median log10 of (final) number of citations of references in "+string_journal+string_plos_field+"<br> papers from "+str(years)+", "+string_age_selection+string_isolated_ref+string_self_ref+string_code_categ+"<br>Number of occurrences: "+str(N_all)#+",   Number plos: "+str(N_plos)
    elif  v1_string ==  'log2_num_cit_ref' :
        fig_title_plot = "Median log2 of (final) number of citations of references in "+string_journal+string_plos_field+"<br> papers from "+str(years)+", "+string_age_selection+string_isolated_ref+string_self_ref+string_code_categ+"<br>Number of occurrences: "+str(N_all)#+",   Number plos: "+str(N_plos)







elif v1_string =='ref_pub_year':
    fig_colorscale = "Viridis"
    fig_title_plot = "Median age of references in "+string_journal+string_plos_field+"<br> papers from "+str(years)+string_isolated_ref+string_self_ref+string_code_categ+"<br>Number of occurrences: "+str(N_all)#+",   Number plos: "+str(N_plos)
    fig_filename = '../plots/annotated-heatmap_median_age_of_references_for_sections_and_fract_subsection_by_citations_of_plos'   
    print ("  No preselection by age of references:",preselection_df3.shape )







elif v1_string =='diff_year_plos_ref':

#     factor_color_rescale =.6  

#     fig_colorscale=[[0.0*factor_color_rescale, '#ffffff'],\
#                            [0.1*factor_color_rescale, '#d9f2d9'],\
#                            [0.2*factor_color_rescale, '#c6ecc6'],\
#                            [0.3*factor_color_rescale, '#b3e6b3'],\
#                            [0.4*factor_color_rescale, '#8cd98c'], \
#                            [0.5*factor_color_rescale, '#66cc66'], \
#                            [0.6*factor_color_rescale, '#53c653'], \
#                            [0.7*factor_color_rescale, '#40bf40'],\
#                            [0.75*factor_color_rescale, '#39ac39'],\
#                            [0.8*factor_color_rescale, '#339933'],\
#                            [0.85*factor_color_rescale, '#2d862d'],\
#                            [0.9*factor_color_rescale, '#267326'],\
#                           [1.0, '#000000']]
    
  
    
    
    fig_colorscale = [[0, '#dcf0d2'], [1, '#205803']]   # if i give it a min and a max colors in HEX, it creates a gradient from one to another
    fig_font_colors = ['#205803', '#dcf0d2']      # same for the annotation of the boxes (to make sure they are readable)
    fig_title_plot = "Median difference between publication year of plos and references in "+string_journal+string_plos_field+" papers from "+str(years)+string_isolated_ref+string_self_ref+string_code_categ+"<br>Number of occurrences: "+str(N_all)#+",   Number plos: "+str(N_plos)
    fig_filename = '../plots/annotated-heatmap_median_age_difference_plos_publ_year_vs_references_for_sections_and_subsect_by_citations_of_plos'

    print ("  No preselection by age of references:",preselection_df3.shape )






print ("\nTot # records included:",len(preselection_df3),"   # number of plos papers:",len(preselection_df3.paper_UT.unique()), "   # unique ref:", len(preselection_df3.reference_UT.unique()),'\n')







#### i get the bins number of citation of the plos papers OJO!!!!! i want the same bins for all papers (so i calculate them before separating into sections but after all the preselections)

#quantiles=sorted(list(df_plos[string_filtering_x].quantile(list_q).to_dict().items())) #mean 10.68 
quantiles=sorted(list(preselection_df3[string_filtering_x].quantile(list_q).to_dict().items())) #mean 10.68 


lista_bins_plos_citations=[]
old_value=0
for item in quantiles:
    try:
        pair=[old_value, int(item[1])]    
    except:  # if it is a nan:
        pair=[old_value, item[1]]

    lista_bins_plos_citations.append(pair)

    try:
        old_value = int(item[1])
    except:
        old_value = item[1]

print (lista_bins_plos_citations)



### i modify the bins to separete the zero-one
# lista_bins_plos_citations[0][0]=2       
# lista_bins_plos_citations = [[0,2]] + lista_bins_plos_citations    
#print (lista_bins_plos_citations)

################################################









lista_titulos_sets = []



lista_sections = ["Introduction","Methods","Results","Discussion"]

cont=0
for item in lista_bins_plos_citations:

    minimo = item[0]
    maximo = item[1]


    preselection_df4 = preselection_df3[(preselection_df3[string_filtering_x] >= minimo)  &  (preselection_df3[string_filtering_x] < maximo)]
    #print ("size of preselection3 (by cit bin plos):",df_select.shape, item)



    x1_All = list(preselection_df4[v1_string])









    for string_section in lista_sections:


        ##### preselection to include only occurences in a section of the paper
        if  string_section == "Introduction":
            section=0
        elif  string_section == "Methods":
            section=1
        elif  string_section == "Results":
            section=2
        elif  string_section == "Discussion":
            section=3





        df_select = preselection_df4[preselection_df4['regex_sect_index']== section]   
        #print ("size of preselection2 (by section):",preselection_df3.shape, string_section)

        x1 = list(df_select[v1_string])       





        if cont ==0:            
            #group=string_section+" Bottom "+str(int(100.*list_q[0]))+"%"  
            group=string_section+" Bottom" 
        elif cont ==1:            
            #group=string_section+" "+str(int(100.*list_q[0]+1))+"%-"+str(int(100.*list_q[-4]))+"%"         
            group=string_section+" Typical"       
        elif cont==2:
             #group=string_section+" "+str(int(100.*list_q[1]+1))+"%-"+str(int(100.*list_q[-3]))+"%"     
             group=string_section+" Good"    
        elif cont==3: 
            #group=string_section+" "+str(int(100.*list_q[2]+1))+"%-"+str(int(100.*list_q[-2]))+"%"    
            group=string_section+" High"
        elif cont==4:
            #group=string_section+" Top "+str(int(100.-100.*list_q[-2]))+"%"    
            group=string_section+" Top"
            
            
        lista_titulos_sets.append(group)    





        ######### i get also quantiles for each cell:    
        list_quantiles_cell=[.25,.5,.75]

        values_quantiles=list(df_select[v1_string].quantile(list_quantiles_cell))#sorted(list(df_select[v1_string].quantile(list_quantiles_cell).to_dict().items()))      

        tupla=values_quantiles + [len(x1)]

        dict_group_quantiles_size[group] = tupla

        dict_group_subset_data[group]=x1



    cont +=1





################ i also add the median values for the section across all data in the preselection
for string_section in lista_sections:


    if  string_section == "Introduction":
        section=0
    elif  string_section == "Methods":
        section=1
    elif  string_section == "Results":
        section=2
    elif  string_section == "Discussion":
        section=3


    df_select = preselection_df3[preselection_df3['regex_sect_index']== section]   

    list_quantiles_cell=[.25,.5,.75]
    values_quantiles=list(df_select[v1_string].quantile(list_quantiles_cell))#sorted(list(df_select[v1_string].quantile(list_quantiles_cell).to_dict().items()))      
    tupla=values_quantiles + [len(df_select)]



#     dict_group_quantiles_size[string_section+" ALL PLOS"]=tupla
#     dict_group_subset_data[string_section+" ALL PLOS"]=x1_All    







########  I create the x, y, z lists of values for the heatmap

lista_y=lista_sections
#lista_x=[" Bottom 30%"," 31% to 60%"," 61% to 90%" ," 91% to 99%"," Top 1%"," ALL"]
#lista_bin_names=[" ALL PLOS"," Bottom "+str(int(100.*list_q[0]))+"%"," "+str(int(100.*list_q[0]+1))+"% to "+str(int(100.*list_q[-4]))+"%",\
              #   " "+str(int(100.*list_q[1]+1))+"% to "+str(int(100.*list_q[-3]))+"%" ," "+str(int(100.*list_q[2]+1))+"% to "+str(int(100.*list_q[-2]))+"%"," Top "+str(int(100.-100.*list_q[-2]))+"%"]


lista_bin_names=[" Bottom"," Typical"," Good"," High"," Top"]






#lista_x=[" Bottom 30%"," 31% to 60%"," 61% to 90%" ," 91% to 99%"," Top 1%"]   

lista_x=lista_bin_names

lista_z25=[]
lista_z50=[]
lista_z75=[]
lista_z_sizes=[]

for x_value in lista_x:    
    aux_lista25=[]
    aux_lista50=[]
    aux_lista75=[]
    aux_lista_sizes=[]

    for y_value in lista_y:       

        llave=y_value+x_value

        try:
            value=int(dict_group_quantiles_size[llave][0])
        except:  # if it is a nan:
            value=dict_group_quantiles_size[llave][0]
        aux_lista25.append(value)




        try:
            value=int(dict_group_quantiles_size[llave][1])
        except:  # if it is a nan:
            value=dict_group_quantiles_size[llave][1]
        aux_lista50.append(value)




        try:
            value=int(dict_group_quantiles_size[llave][2])
        except:  # if it is a nan:
            value=dict_group_quantiles_size[llave][2]
        aux_lista75.append(value)




        value_size=dict_group_quantiles_size[llave][3]
        aux_lista_sizes.append(value_size)


        #print (y_value," ",x_value, value, value_size)
    lista_z25.append(aux_lista25)
    lista_z50.append(aux_lista50)
    lista_z75.append(aux_lista75)




    lista_z_sizes.append(aux_lista_sizes)



# print ("lista values 25%-quantile:",lista_z25)
# print ("lista sizes",lista_z_sizes)





lista_text_z=[]
for i in range(len(lista_z_sizes)):
    aux=[]
    for j in range(len(lista_z_sizes[0])):
        #value=str(lista_z25[i][j])+"-<b>"+str(lista_z50[i][j])+"</b>-"+str(lista_z75[i][j])+"<br>("+str(lista_z_sizes[i][j])+")"            #"Median:"+str(lista_z[i][j])+"<br> N:"+str(lista_z_sizes[i][j])
        value=str(lista_z25[i][j])+"-<b>"+str(lista_z50[i][j])+"</b>-"+str(lista_z75[i][j])+"<br>("+str(format(lista_z_sizes[i][j], ',d'))+")"            #"Median:"+str(lista_z[i][j])+"<br> N:"+str(lista_z_sizes[i][j])



        aux.append(value)
    lista_text_z.append(aux)
# print (lista_text_z)





##### using a different library
path=   '/home/staff/julia/at_Northwestern/In_Text_Citations/In-Text-Citations-New/plots/'


fig = ff.create_annotated_heatmap(z=lista_z50, x=lista_sections, y=lista_bin_names, annotation_text=lista_text_z, colorscale=fig_colorscale, font_colors=fig_font_colors,showscale=True, colorbar=dict(title=colorbar_string, titleside='right' ,ticks='outside'),)#, reversescale=True)

#fig = ff.create_annotated_heatmap(z=lista_z50, x=lista_sections, y=lista_bin_names,  colorscale=fig_colorscale, font_colors=fig_font_colors,showscale=True)#, reversescale=True)
fig.layout.title = ""# fig_title_plot

fig['layout']['xaxis']['side'] = 'bottom'
fig.layout.xaxis.update({'title': 'Section'})


fig.layout.yaxis.update({'title': 'Impact Group'})
if v1_string ==  'cite_count'  :
    if string_references_age == "young":  
        fig.layout.yaxis.update({'title': ''})




font_gral=25  # 20 if i wanna see it on the browser, 40 if i care about the png output
fig['layout']['font']['size'] = font_gral

      
if v1_string ==  'cite_count'  :
    if string_references_age == "young":  
        #fig.layout.update({'title': '$d, r \\text{ (solar radius)}$'})
        fig['layout']['title'] = "Young references"
    elif string_references_age == "old":  
        fig.layout.update({'title': 'Old references'})

    fig.layout.update({'font': dict(size=25)})

# fig.layout.yaxis.update({'title': 'Citation percentile of plos paper'})




font_gral=55  # 20 if i wanna see it on the browser, 40 if i care about the png output
fig['layout']['font']['size'] = font_gral


# Altering x axis
#fig['layout']['xaxis']['tickfont']['family'] = 'Gill Sans MT'
fig['layout']['xaxis']['tickangle'] = 0
fig['layout']['yaxis']['tickangle'] = -90
fig['layout']['xaxis']['titlefont']['size'] = font_gral + 20
fig['layout']['yaxis']['titlefont']['size'] = font_gral

fig['layout']['xaxis']['tickfont']['size'] = font_gral -7 
fig['layout']['yaxis']['tickfont']['size'] = font_gral -15




fig['layout']['margin']=dict(
        l=200,
       # r=50,
        b=150,
        t=top_space,
        pad=15
    )





#  annotations=[
#         dict(
#             x=-.05,
#             y=1.05,
#             xref='paper',
#             yref='paper',
#             text=text_abc,
#             showarrow=False,
#             font=dict(               
#                 size=80,  
#                 color='black'
#             ),
           
#         )
#     ],


offline.plot(fig, auto_open=True, image = 'png', image_filename=fig_filename ,image_width=2000, image_height=1200, filename=fig_filename+'.html', validate=True)





# old, age >=15 years; Tot # records included: 88209    # number of plos papers: 12895    # unique ref: 61867 
# young, age <=2 years;  Tot # records included: 104113    # number of plos papers: 13442    # unique ref: 66644 












###########################################
######################################################3
########################################################


##############  I RUN THE TESTS FOR THE COMPARISON OF ALL PAIRS OF SUB-SETS:  (all pair-wise comparison cells in figure 2b, and 3a, 2b)

#####  mann-whitney U   test



lista_indeces = [[1,1],[1,2],[1,3],[1,4],\
                 [2,1],[2,2],[2,3],[2,4],\
                 [3,1],[3,2],[3,3],[3,4],\
                 [4,1],[4,2],[4,3],[4,4],\
                 [5,1],[5,2],[5,3],[5,4]]

#fig_colorscale=  [ [0., '#f2f2f2'], [threshold_zero,'#cce6ff'], [0.9999, '#0059b3'], [1.,'#bdbdbd']]# '#ffffff']]  0: white,  .99999: blue,  1: grey
fig_colorscale=  [ [0., '#0059b3'], [.5,'#c7dcf1'], [1.,'#bdbdbd']] #  0 or anything significant: blue,   .5 or anithing NON signif: light-blue,     1: grey






lista_bin_names = ["Bottom","Typical","Good","High","Top"]
lista_sections  =  ['Intro', 'Methods', 'Results', 'Discussion']






tot_rows = 5
tot_cols = 4  
    
test = "MW"  # KS to test if the distributions are different or  MW for testing just whether the medians are different 
    

    
    
list_keys_macro = ['Introduction Top',    'Methods Top',    'Results Top',   'Discussion Top', \
                   'Introduction High',   'Methods High',  'Results High',  'Discussion High', \
                   'Introduction Good',   'Methods Good',   'Results Good',  'Discussion Good', \
                   'Introduction Typical', 'Methods Typical', 'Results Typical', 'Discussion Typical', \
                   'Introduction Bottom',  'Methods Bottom',  'Results Bottom', 'Discussion Bottom']
    
      
list_keys_heatmap = ['Introduction Bottom', 'Methods Bottom',  'Results Bottom', 'Discussion Bottom',\
                     'Introduction Typical', 'Methods Typical', 'Results Typical', 'Discussion Typical',\
                     'Introduction Good',   'Methods Good',   'Results Good',  'Discussion Good', \
                     'Introduction High',   'Methods High',  'Results High',  'Discussion High', \
                     'Introduction Top',   'Methods Top',    'Results Top',   'Discussion Top'  ]


### i apply the bonferroni correction:  new p-_value_threshold  required for significance = old_p_value /Number of comparisons = 0.001 / (20*19/2)     
    
threshold_zero = 0.0001 / (float(len(list_keys_macro))*float(len(list_keys_macro)-1)/2.)    # to round up to zero the very small p_values



# lista_titulos_sets  (from the previous cell)

# ['Introduction Bottom',
#  'Methods Bottom',
#  'Results Bottom',
#  'Discussion Bottom',
#  'Introduction Typical',
#  'Methods Typical',
#  'Results Typical',
#  'Discussion Typical',
#  'Introduction Good',
#  'Methods Good',
#  'Results Good',
#  'Discussion Good',
#  'Introduction High',
#  'Methods High',
#  'Results High',
#  'Discussion High',
#  'Introduction Top',
#  'Methods Top',
#  'Results Top',
#  'Discussion Top']






lista_tot_datos=[]

total_cont =0
for i in list_keys_macro:
    lista_listas=[]
    aux_lista=[]
    cont=1
   
    for j in list_keys_heatmap:
             
        set1 = dict_group_subset_data[i]
        set2 = dict_group_subset_data[j]
        
        if test == "KS":
            p_value = stats.ks_2samp(set1, set2)[1] 
        elif test == "MW":
            p_value = stats.mannwhitneyu(set1, set2,  alternative='two-sided')[1]  
        
        
        if p_value <= threshold_zero:  #i round up to zero the very small p_values
                p_value =0.
                
        else:
            p_value = .5  ### I ONLY CARFE ABOUT WHETGER IT IS SIGNIFICANT OR NOT, I DONT CARE ABOUT THE EXACT P-VALUE ONCE IT IS NOT SIGNIFICANT
                
        if i == j:  # i single out manually the self comparison
            p_value=1.001  
            
            
            
        aux_lista.append(p_value)
        
        cont +=1
        
        if cont > tot_cols:
            lista_listas.append(aux_lista)                  
            aux_lista=[]
            cont=1
            
    
    
    total_cont +=1
    lista_tot_datos.append(lista_listas)
    
    
   


##############################

### I create an empty plot
fig_macro = None
fig_macro = tools.make_subplots(rows=tot_rows, cols=tot_cols, shared_xaxes=True, shared_yaxes=True, vertical_spacing = 0.01, horizontal_spacing = 0.01,   )


for i in range(len(lista_tot_datos)):
    datos = lista_tot_datos[i]
    
    cont_rows = lista_indeces[i][0]
    cont_cols = lista_indeces[i][1]
    
    trace1 = go.Heatmap(z=datos,
                       x=lista_sections,
                       y=lista_bin_names,                        
                       colorscale = fig_colorscale,
                       showscale=False,)
                        #reversescale=True, )#    )

   



    
    # i add each plot
    fig_macro.append_trace(trace1, cont_rows, cont_cols)


 



fontsize=32 
fig_macro['layout']['font'].update({'size': fontsize})






# fig_macro['layout']['margin'].update({'l': 250})   # i add extra space to the margins of the plots
# fig_macro['layout']['margin'].update({'b': 150})  
# fig_macro['layout']['margin'].update({'t': 50})   



lista_bin_names = ["Bottom","Typical","Good","High","Top"]
lista_sections  =  ['Introduction', 'Methods', 'Results', 'Discussion']



# i use the axis title of the small plots as values of the axis of the macro plot
fig_macro['layout']['xaxis1'].update(title=lista_sections[0])  
fig_macro['layout']['xaxis2'].update(title=lista_sections[1])
fig_macro['layout']['xaxis3'].update(title=lista_sections[2])
fig_macro['layout']['xaxis4'].update(title=lista_sections[3])


fig_macro['layout']['yaxis1'].update(title=lista_bin_names[4])   # ojo con el orden aqui!!
fig_macro['layout']['yaxis2'].update(title=lista_bin_names[3])
fig_macro['layout']['yaxis3'].update(title=lista_bin_names[2])
fig_macro['layout']['yaxis4'].update(title=lista_bin_names[1])
fig_macro['layout']['yaxis5'].update(title=lista_bin_names[0])







#  annotations=[
#         dict(
#             x=-.05,
#             y=1.05,
#             xref='paper',
#             yref='paper',
#             text='(b)',
#             showarrow=False,
#             font=dict(               
#                 size=80,  
#                 color='black'
#             ),
           
#         )
#     ],
   
         
         


    

offline.plot(fig_macro, auto_open=True, image = 'png', image_filename='multiplot_comparisons' ,image_width=3000, image_height=2200,
              filename='../plots/multiplot_comparisons.html', validate=True)





original size: (5787630, 34)
[2011]
size of preselection1 (by plos years): (564251, 34)
size of preselection1 (by isolated/group ref): (564251, 34) 
 size of preselection2 (by plos journal): (564251, 34) 
 size of preselection2 (by plos field): (564251, 34) 
  size of preselection3 (only young references): (183640, 34) only old references from <=2001

Tot # records included: 183640    # number of plos papers: 14028    # unique ref: 123212 

[[0, 8], [8, 16], [16, 35], [35, 84], [84, 856]]
This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]  [ (1,3) x3,y1 ]  [ (1,4) x4,y1 ]
[ (2,1) x1,y2 ]  [ (2,2) x2,y2 ]  [ (2,3) x3,y2 ]  [ (2,4) x4,y2 ]
[ (3,1) x1,y3 ]  [ (3,2) x2,y3 ]  [ (3,3) x3,y3 ]  [ (3,4) x4,y3 ]
[ (4,1) x1,y4 ]  [ (4,2) x2,y4 ]  [ (4,3) x3,y4 ]  [ (4,4) x4,y4 ]
[ (5,1) x1,y5 ]  [ (5,2) x2,y5 ]  [ (5,3) x3,y5 ]  [ (5,4) x4,y5 ]



'file:///home/staff/julia/at_Northwestern/In_Text_Citations/In-Text-Citations-New/plots/multiplot_comparisons.html'